In [1]:
# -*- coding: utf-8 -*-
"""
Created on 01/14, 2023
@author: WillyF

"""

# https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
# https://opendata.cwb.gov.tw/dataset/forecast/F-C0032-001
# https://opendata.cwb.gov.tw/dist/opendata-swagger.html?urls.primaryName=openAPI#/%E9%A0%90%E5%A0%B1/get_v1_rest_datastore_F_C0032_001
# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
# https://stackoverflow.com/questions/31306741/unmelt-pandas-dataframe
# https://medium.com/%E6%95%B8%E6%93%9A%E4%B8%8D%E6%AD%A2-not-only-data/pandas-%E5%BF%AB%E9%80%9F%E7%9E%AD%E8%A7%A3-pivot-table-%E8%88%87%E6%87%89%E7%94%A8-21e4c37b9216



'\nCreated on 01/14, 2023\n@author: WillyF\n\n'

In [2]:
import pandas as pd
# import urllib
import urllib.request
import json
import numpy as np
# from glob import glob
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

In [3]:
data_source = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=CWB-3D27458E-8F30-4BDF-9589-6DD4A337AA74&format=JSON&locationName=&elementName=&sort=time"
json_url = urllib.request.urlopen(data_source)
data = json.loads(json_url.read())
data

{'success': 'true',
 'result': {'resource_id': 'F-C0032-001',
  'fields': [{'id': 'datasetDescription', 'type': 'String'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'},
   {'id': 'parameterUnit', 'type': 'String'},
   {'id': 'startTime', 'type': 'Timestamp'},
   {'id': 'endTime', 'type': 'Timestamp'}]},
 'records': {'datasetDescription': '三十六小時天氣預報',
  'location': [{'locationName': '嘉義縣',
    'weatherElement': [{'elementName': 'Wx',
      'time': [{'startTime': '2023-02-05 12:00:00',
        'endTime': '2023-02-05 18:00:00',
        'parameter': {'parameterName': '陰時多雲', 'parameterValue': '6'}},
       {'startTime': '2023-02-05 18:00:00',
        'endTime': '2023-02-06 06:00:00',
        'parameter': {'parameterName': '陰時多雲', 'parameterValue': '6'}},
       {'startTime': '2023-02-06 06:00:00',
        'endTime': '2023-02-06 18:00:00',
        'parameter': {'parameterName': '陰時多雲', 'parameterValu

In [4]:
data["records"]

{'datasetDescription': '三十六小時天氣預報',
 'location': [{'locationName': '嘉義縣',
   'weatherElement': [{'elementName': 'Wx',
     'time': [{'startTime': '2023-02-05 12:00:00',
       'endTime': '2023-02-05 18:00:00',
       'parameter': {'parameterName': '陰時多雲', 'parameterValue': '6'}},
      {'startTime': '2023-02-05 18:00:00',
       'endTime': '2023-02-06 06:00:00',
       'parameter': {'parameterName': '陰時多雲', 'parameterValue': '6'}},
      {'startTime': '2023-02-06 06:00:00',
       'endTime': '2023-02-06 18:00:00',
       'parameter': {'parameterName': '陰時多雲', 'parameterValue': '6'}}]},
    {'elementName': 'PoP',
     'time': [{'startTime': '2023-02-05 12:00:00',
       'endTime': '2023-02-05 18:00:00',
       'parameter': {'parameterName': '20', 'parameterUnit': '百分比'}},
      {'startTime': '2023-02-05 18:00:00',
       'endTime': '2023-02-06 06:00:00',
       'parameter': {'parameterName': '20', 'parameterUnit': '百分比'}},
      {'startTime': '2023-02-06 06:00:00',
       'endTime': '20

In [28]:
data_df = pd.json_normalize(data["records"],
                            record_path = ["location","weatherElement","time"],
                            meta = [['location',"locationName"],
                                    ['location',"weatherElement","elementName"],
                                    'datasetDescription',
                                   ],
                           )
data_df.columns = [x.split(".")[-1] for x in data_df.columns]
data_df["parameterUnit"].replace("百分比","%", inplace=True)
data_df["parameterUnit"].replace("C","°C", inplace=True)
data_df["parameterUnit"].replace(np.nan,"", inplace=True)
data_df["elementName"].replace("Wx","Weather Forcast", inplace=True)
data_df["elementName"].replace("PoP","Probability of Precipitation", inplace=True)
data_df["elementName"].replace("MinT","Min Temperature", inplace=True)
data_df["elementName"].replace("MaxT","Max Temperature", inplace=True)
data_df["elementName"].replace("CI","Comfort Index", inplace=True)
data_df["Parameter"] = data_df["parameterName"]+data_df["parameterUnit"]
data_df["startTime"] = data_df["startTime"].apply(lambda x: x.replace("-","/")[:-3])
data_df["endTime"] = data_df["endTime"].apply(lambda x: x.replace("-","/")[:-3])


data_df

,startTime,endTime,parameterName,parameterValue,parameterUnit,locationName,elementName,datasetDescription,Parameter
0,2023/02/05 12:00,2023/02/05 18:00,陰時多雲,6,,嘉義縣,Weather Forcast,三十六小時天氣預報,陰時多雲
1,2023/02/05 18:00,2023/02/06 06:00,陰時多雲,6,,嘉義縣,Weather Forcast,三十六小時天氣預報,陰時多雲
2,2023/02/06 06:00,2023/02/06 18:00,陰時多雲,6,,嘉義縣,Weather Forcast,三十六小時天氣預報,陰時多雲
3,2023/02/05 12:00,2023/02/05 18:00,20,NaN,%,嘉義縣,Probability of Precipitation,三十六小時天氣預報,20%
4,2023/02/05 18:00,2023/02/06 06:00,20,NaN,%,嘉義縣,Probability of Precipitation,三十六小時天氣預報,20%
5,2023/02/06 06:00,2023/02/06 18:00,20,NaN,%,嘉義縣,Probability of Precipitation,三十六小時天氣預報,20%
6,2023/02/05 12:00,2023/02/05 18:00,20,NaN,°C,嘉義縣,Min Temperature,三十六小時天氣預報,20°C
7,2023/02/05 18:00,2023/02/06 06:00,17,NaN,°C,嘉義縣,Min Temperature,三十六小時天氣預報,17°C
8,2023/02/06 06:00,2023/02/06 18:00,17,NaN,°C,嘉義縣,Min Temperature,三十六小時天氣預報,17°C
9,2023/02/05 12:00,2023/02/05 18:00,稍有寒意至舒適,NaN,,嘉義縣,Comfort Index,三十六小時天氣預報,稍有寒意至舒適


In [17]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   startTime           330 non-null    object
 1   endTime             330 non-null    object
 2   parameterName       330 non-null    object
 3   parameterValue      66 non-null     object
 4   parameterUnit       330 non-null    object
 5   locationName        330 non-null    object
 6   elementName         330 non-null    object
 7   datasetDescription  330 non-null    object
 8   Parameter           330 non-null    object
dtypes: object(9)
memory usage: 23.3+ KB


In [32]:
data_df_pivot = pd.pivot_table(data_df,
                               index=["startTime","endTime", 'locationName'], 
                               columns=["elementName"], 
                               values=["Parameter"],
                               aggfunc= lambda x: x).reset_index()
data_df_pivot.columns = ["startTime","endTime","locationName","Comfort Index",
                         "Max Temperature","Min Temperature","Probability of Precipitation","Weather Forcast"]
data_df_pivot = data_df_pivot.sort_values(by=["locationName","startTime","endTime"]).reset_index(drop=True)
data_df_pivot["Period"] = data_df_pivot["startTime"].str.cat(data_df_pivot["endTime"], sep =" ~ ")
data_df_pivot["Temperature"] = data_df_pivot["Max Temperature"].str.cat(data_df_pivot["Min Temperature"], sep =" ~ ")
data_df_pivot = data_df_pivot.rename(columns={"locationName": "Location", "Probability of Precipitation": "PoP", "Weather Forcast": "Weather FCST"}, errors="raise")

data_df_pivot

,startTime,endTime,Location,Comfort Index,Max Temperature,Min Temperature,PoP,Weather FCST,Period,Temperature
0,2023/02/05 12:00,2023/02/05 18:00,南投縣,稍有寒意至舒適,22°C,20°C,30%,陰短暫雨,2023/02/05 12:00 ~ 2023/02/05 18:00,22°C ~ 20°C
1,2023/02/05 18:00,2023/02/06 06:00,南投縣,稍有寒意,20°C,17°C,20%,陰天,2023/02/05 18:00 ~ 2023/02/06 06:00,20°C ~ 17°C
2,2023/02/06 06:00,2023/02/06 18:00,南投縣,稍有寒意至舒適,24°C,17°C,20%,陰時多雲,2023/02/06 06:00 ~ 2023/02/06 18:00,24°C ~ 17°C
3,2023/02/05 12:00,2023/02/05 18:00,嘉義市,舒適,24°C,21°C,20%,陰天,2023/02/05 12:00 ~ 2023/02/05 18:00,24°C ~ 21°C
4,2023/02/05 18:00,2023/02/06 06:00,嘉義市,稍有寒意至舒適,21°C,17°C,20%,陰時多雲,2023/02/05 18:00 ~ 2023/02/06 06:00,21°C ~ 17°C
5,2023/02/06 06:00,2023/02/06 18:00,嘉義市,稍有寒意至舒適,26°C,17°C,20%,多雲,2023/02/06 06:00 ~ 2023/02/06 18:00,26°C ~ 17°C
6,2023/02/05 12:00,2023/02/05 18:00,嘉義縣,稍有寒意至舒適,22°C,20°C,20%,陰時多雲,2023/02/05 12:00 ~ 2023/02/05 18:00,22°C ~ 20°C
7,2023/02/05 18:00,2023/02/06 06:00,嘉義縣,稍有寒意,20°C,17°C,20%,陰時多雲,2023/02/05 18:00 ~ 2023/02/06 06:00,20°C ~ 17°C
8,2023/02/06 06:00,2023/02/06 18:00,嘉義縣,稍有寒意至舒適,24°C,17°C,20%,陰時多雲,2023/02/06 06:00 ~ 2023/02/06 18:00,24°C ~ 17°C
9,2023/02/05 12:00,2023/02/05 18:00,基隆市,稍有寒意,19°C,18°C,90%,陰短暫雨,2023/02/05 12:00 ~ 2023/02/05 18:00,19°C ~ 18°C


In [9]:
sorted(set(data_df_pivot["locationName"]))

['南投縣',
 '嘉義市',
 '嘉義縣',
 '基隆市',
 '宜蘭縣',
 '屏東縣',
 '彰化縣',
 '新北市',
 '新竹市',
 '新竹縣',
 '桃園市',
 '澎湖縣',
 '臺中市',
 '臺北市',
 '臺南市',
 '臺東縣',
 '花蓮縣',
 '苗栗縣',
 '連江縣',
 '金門縣',
 '雲林縣',
 '高雄市']

In [ ]:
sorted(set(data_df_pivot["locationName"]))